In [1]:
#Sytem
import sys,os
import numpy as np
import pandas as pd
import warnings
import time
import random
import IPython
warnings.filterwarnings('ignore')

#Visualidation
import sweetviz as sv
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

#EDA
import sweetviz as sv
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder



#model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from skopt import BayesSearchCV




print('-'*40)

print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))


print("scikit-learn version: {}". format(sklearn.__version__))
print('-'*40)


----------------------------------------
Python version: 3.9.10 (main, Feb 22 2022, 13:54:07) 
[GCC 11.2.0]
pandas version: 1.3.5
matplotlib version: 3.5.1
NumPy version: 1.21.5
scikit-learn version: 1.1.1
----------------------------------------


# Info

In [2]:
Target = "Perished"
PATH_LOG = "./log.csv"
PATH_df = "./df"
ID = 10
SEED = 42

# Import Dataset

In [5]:
df_train = pd.read_csv("../data/input/train.csv")
df_test = pd.read_csv("../data/input/test.csv")
#log = pd.read_csv(PATH_LOG)
data = pd.concat([df_train,df_test],sort=False)
print('-'*25)
print("Shape of Train Data\n[row :{},column :{}]".format(
    df_train.shape[0], df_train.shape[1]))
print('-'*25)
print("Shape of Test Data\n[row :{},column :{}]".format(
    df_test.shape[0], df_test.shape[1]))
print('-'*25)

-------------------------
Shape of Train Data
[row :891,column :12]
-------------------------
Shape of Test Data
[row :418,column :11]
-------------------------


In [64]:
log.tail(2)

,Modelname,STrain,SValid,Pclass,Sex,Age,SibSp,Parch,Fare,C,...,Age_Code_N,SPublic,Title_Code,Embarked_Code,Search,CV,Bestparams,Title,FareBin,Fare_Code
26,RandomForestClassifier,0.907303,0.837989,o,o,o,o,o,o,o,...,3,x,x,x,BS,3,"OrderedDict([('bootstrap', True), ('max_depth'...",x,x,x
27,RandomForestClassifier,0.886236,0.888268,o,o,o,o,o,o,o,...,3,0.788,x,x,BS,3,"OrderedDict([('bootstrap', True), ('max_depth'...",x,x,x


# EDA

## Report

In [65]:
# repoort = sv.analyze(df_train,target_feat="Perished")
# repoort.show_html("SWEETVIZ_REPORT.html")

## Feature Engineering

In [6]:
label = LabelEncoder()
ohe = OneHotEncoder()

### Title

In [7]:
data['Title'] = data['Name'].str.split(
        ", ", expand=True)[1].str.split(".", expand=True)[0]

title_names = (data.Title.value_counts() < 10)
data["Title"] = data["Title"].apply(lambda x: "other" if title_names.loc[x] == True else x)

### Age

In [8]:
#for title in data.Title.unique():
#    mean = data.loc[(data.Title.values == title),"Age"].mean()
#    data.loc[(data.Title.values == title) & (data.Age.isnull()),"Age"] = mean


### Fare

In [9]:
faremean = data.Fare.mean()
data["Fare"].fillna(faremean,inplace=True)
data['FareBin'] = pd.qcut(data['Fare'], 10)
data['Fare_Code'] = label.fit_transform(data['FareBin'])

### Encoding

In [10]:
N_Age_Code = 3
data.replace({'Sex':{'male':0,'female':1}},inplace=True)
data['Age_Code'] = label.fit_transform(pd.qcut(data['Age'], N_Age_Code))
data['Embarked_Code'] = label.fit_transform(data['Embarked'])
data["Title_Code"] = label.fit_transform(data["Title"])

### Embarked

In [11]:
data["Embarked"].fillna('S',inplace=True)
embarked_ohe = pd.get_dummies(data.Embarked)
data = pd.concat([data, embarked_ohe], axis=1)

In [12]:
data["Family"] = data["SibSp"] + data["Parch"]

,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Title,FareBin,Fare_Code,Age_Code,Embarked_Code,Title_Code,C,Q,S,Family
0,1,1.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,...,Mr,"(-0.001, 7.57]",0,0,2,2,0,0,1,1
1,2,0.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,...,Mrs,"(41.579, 78.02]",8,2,0,3,1,0,0,1
2,3,0.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,...,Miss,"(7.854, 8.05]",2,1,2,1,0,0,1,0
3,4,0.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,...,Mrs,"(41.579, 78.02]",8,2,2,3,0,0,1,1
4,5,1.0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,...,Mr,"(7.854, 8.05]",2,2,2,2,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",0,NaN,0,0,A.5. 3236,8.0500,...,Mr,"(7.854, 8.05]",2,3,2,2,0,0,1,0
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",1,39.0,0,0,PC 17758,108.9000,...,other,"(78.02, 512.329]",9,2,0,4,1,0,0,0
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",0,38.5,0,0,SOTON/O.Q. 3101262,7.2500,...,Mr,"(-0.001, 7.57]",0,2,2,2,0,0,1,0
416,1308,NaN,3,"Ware, Mr. Frederick",0,NaN,0,0,359309,8.0500,...,Mr,"(7.854, 8.05]",2,3,2,2,0,0,1,0


In [72]:
drop_columns = ["Name","Ticket","Cabin","Embarked","Title",'Embarked_Code','FareBin',"Fare_Code"]
dataset = data.drop(columns=drop_columns)
train = dataset[:len(df_train)]
test = dataset[len(df_train):]

In [74]:
data.Age.isnull().sum()

263

In [75]:
test.head()

,PassengerId,Perished,Pclass,Sex,Age,SibSp,Parch,Fare,Age_Code,C,Q,S
0,892,NaN,3,0,34.5,0,0,7.8292,2,0,1,0
1,893,NaN,3,1,47.0,1,0,7.0000,2,0,0,1
2,894,NaN,2,0,62.0,0,0,9.6875,2,0,1,0
3,895,NaN,3,0,27.0,0,0,8.6625,1,0,0,1
4,896,NaN,3,1,22.0,1,1,12.2875,0,0,0,1


In [76]:
train.to_csv(os.path.join(PATH_df, "EDA_"+str(ID)+"_train.csv"), index_label=False)
test.to_csv(os.path.join(PATH_df,"EDA_"+str(ID)+ "_test.csv"), index_label=False)

# Model

In [77]:
X = train.iloc[:, 2:].values
y = train.iloc[:, 1].values
X_test = test.iloc[:, 2:].values

In [78]:
test.isnull().sum()

PassengerId      0
Perished       418
Pclass           0
Sex              0
Age             86
SibSp            0
Parch            0
Fare             0
Age_Code         0
C                0
Q                0
S                0
dtype: int64

In [61]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
base_model = RandomForestClassifier(n_estimators = 10, random_state = SEED)
base_model.fit(X_train, y_train)
print('Train Score: {}'.format(round(base_model.score(X_train, y_train), 4)))
print(' Valid Score: {}'.format(round(base_model.score(X_valid, y_valid), 4)))

ValueError: Input y contains NaN.

In [64]:
Modelname = "RFC"
rfc = RandomForestClassifier()
CV = 3
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
param_grid = {
    'n_estimators':n_estimators,
    'max_features':max_features,
    'max_depth':max_depth,
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf,
    'bootstrap':bootstrap
}
print(param_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


## Param Search

### Grid Search

In [ ]:
# Search = "GS"
# rfc_cv = GridSearchCV(estimator=rfc,param_grid=param_grid,scoring='accuracy',cv=CV,verbose=2,n_jobs=-1)
# rfc_cv.fit(X_train,y_train)

### Random Search

In [ ]:
Search = "RS"
rfc_cv = RandomizedSearchCV(estimator=rfc,param_distributions=param_grid,n_iter=100,cv=CV,verbose=1,random_state=SEED,n_jobs=-1)
rfc_cv.fit(X_train,y_train)

### Bays Search

In [65]:
Search = "BS"
rfc_cv = BayesSearchCV(estimator=rfc,search_spaces=param_grid,n_iter=100,cv=CV,verbose =2)
rfc_cv.fit(X,y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=800; total time=   0.8s
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=800; total time=   0.8s
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=800; total time=   0.8s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1600; total time=   1.6s
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1600; total time=   1.6s
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1600; total time=   1.6s
Fitting 3 folds for each of 1 candidates, totalling 3 

BayesSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
              search_spaces={'bootstrap': [True, False],
                             'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                           100, 110, None],
                             'max_features': ['auto', 'sqrt'],
                             'min_samples_leaf': [1, 2, 4],
                             'min_samples_split': [2, 5, 10],
                             'n_estimators': [200, 400, 600, 800, 1000, 1200,
                                              1400, 1600, 1800, 2000]},
              verbose=2)

In [66]:
print(rfc_cv.best_params_)

OrderedDict([('bootstrap', True), ('max_depth', 30), ('max_features', 'sqrt'), ('min_samples_leaf', 4), ('min_samples_split', 10), ('n_estimators', 1800)])


In [67]:
best_model = rfc_cv.best_estimator_
print(best_model)
print('Train Score: {}'.format(round(best_model.score(X_train, y_train), 4)))
print(' Valid Score: {}'.format(round(best_model.score(X_valid, y_valid), 4)))

RandomForestClassifier(max_depth=30, min_samples_leaf=4, min_samples_split=10,
                       n_estimators=1800)
Train Score: 0.8862
 Test Score: 0.8883


In [68]:
bestparam = str(rfc_cv.best_params_)
STrain = best_model.score(X_train, y_train)
SValid  = best_model.score(X_valid, y_valid)

## Predict

In [69]:
pred = rfc_cv.predict(X_test)

## Submit

In [70]:
PATH_sub =  ("./sub/"+str(ID)+"_submission_"+Modelname+".csv")
print(PATH_sub)
submission_ex = pd.read_csv("gender_submission.csv")
submission_df = pd.DataFrame(columns=['PassengerId', Target])
submission_df['PassengerId'] = df_test['PassengerId']
submission_df[Target] = pred
submission_df.reset_index(drop=True, inplace=True)
submission_df.to_csv(PATH_sub, header=True, index=False)
print("Submission Format:{} ".format(submission_ex.shape))
print("My Sumbisison Format:{}".format(submission_df.shape))
submission_df.head(5)

./sub/10_submission_RandomForestClassifier.csv
Submission Format:(418, 2) 
My Sumbisison Format:(418, 2)


,PassengerId,Perished
0,892,1.0
1,893,1.0
2,894,1.0
3,895,1.0
4,896,1.0


## Log

In [71]:
Row  =  len(log)
log.loc[Row] = "x"
log.loc[Row,"Modelname"] = Modelname
log.loc[Row,"Search"] = Search
log.loc[Row,"CV"] = CV
log.loc[Row, "Bestparams"] = bestparam 
log.loc[Row,"STrain"] = STrain
log.loc[Row,"SValid"] = SValid
log.loc[Row,"Age_Code_N"] = N_Age_Code
use_columns = test.columns.drop(["PassengerId","Perished"])
for col in use_columns:
    log.loc[Row,col] = "o"
log.fillna("x",inplace=True)
# if PublicScore is knowed
#log.loc[Row,"SPublic"] = 0.791

In [72]:
log.to_csv(PATH_LOG,index_label=False)
log.tail()

,Modelname,STrain,SValid,Pclass,Sex,Age,SibSp,Parch,Fare,C,...,Age_Code_N,SPublic,Title_Code,Embarked_Code,Search,CV,Bestparams,Title,FareBin,Fare_Code
23,RandomForestClassifier,0.889045,0.832402,o,o,o,o,o,o,o,...,3,x,x,x,RS,3,"{'n_estimators': 1000, 'min_samples_split': 10...",x,x,o
24,RandomForestClassifier,0.903090,0.843575,o,o,o,o,o,o,o,...,3,x,x,x,BS,3,"OrderedDict([('bootstrap', True), ('max_depth'...",x,x,x
25,RandomForestClassifier,0.905899,0.843575,o,o,o,o,o,o,o,...,3,x,x,x,RS,3,"{'n_estimators': 600, 'min_samples_split': 10,...",x,x,x
26,RandomForestClassifier,0.907303,0.837989,o,o,o,o,o,o,o,...,3,x,x,x,BS,3,"OrderedDict([('bootstrap', True), ('max_depth'...",x,x,x
27,RandomForestClassifier,0.886236,0.888268,o,o,o,o,o,o,o,...,3,x,x,x,BS,3,"OrderedDict([('bootstrap', True), ('max_depth'...",x,x,x
